# Building a Hash function

## Introduction
goal of this jupyter notebook is to build a hash function for educational purposes. Speifically we will build the SHA256 hash function, usign the [RFC6234](https://datatracker.ietf.org/doc/html/rfc6234) standard. We will compare the result of our function with the result of the hashlib library. 

Intead of the RFC, it's possible to use the [FIPS 180-2](https://csrc.nist.gov/csrc/media/publications/fips/180/2/archive/2002-08-01/documents/fips180-2.pdf) standard. The hashlib library uses definition in FIPS 180-2: RFC 6234 is an adaptation of FIPS 180-2 for use in the Internet Engineering Task Force (IETF) standards process as explainded in the RFC itself. One of the advantages of using the FIPS 180-2 standard is that it's possible to find a lot of examples in the document, which is not the case for the RFC.

The hashlib library uses definition in FIPS 180-2: RFC 6234 is an adaptation of FIPS 180-2 for use in the Internet Engineering Task Force (IETF) standards process as explainded in the RFC itself:

>  Most of the text herein was adapted by the authors from FIPS 180-2.

Let's create a test vector to compare the result of our function with the result of the hashlib library.

In [716]:
import hashlib
import random

# create 10 random strings, convert to bytes, and hash them
preimage_test = []
hash_test = []
for i in range(0, 100,20):
    s = ""
    for j in range(1,i+1): # note an empty string is a valid preimage
        # create random hex string
        s += random.choice("0123456789abcdef")
    # print(s)

    # convert to bytes and add to list
    b = bytes.fromhex(s)
    preimage_test.append(b)

    # hash the bytes and add to list
    h = hashlib.sha256(b)
    hash_test.append(h.hexdigest())

# print the results
for i in range(5):
    print(preimage_test[i].hex(), " --> ", hash_test[i])
    print()

  -->  e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855

906a2ca2051bbb2d29eb  -->  3e9009fcc7b713addcb737a27d83e8659fd34bd17325513207e75995aac29f1c

e5c927061df2ee76538620b289c88fa6c91f1dbe  -->  56e1cea6b40118cd1ecfae70cf29d275bc9bb2b45e6992739d7892e993482134

4f64b6a3d046ddb79a366c4c0b2d97b417d2c55280cd9b8ca544a295d2bf  -->  0e5e2bb5390933c316000192e672cbfcf40a59e2f35bf52c39b79b5ff7e0e5d4

69d78264a1cfaf312eda0cca886bcb4939b77d281a3f48619143397d1c42a8f9e31898076c0d7424  -->  460b27e90573481752b3e1aff05fa7532aeb96a80c9764fbf658ce0d17eab778



## Notation

we create ome helper functons to recreate the notation used in the RFC6234 standard. We will use the notation from the standard to make it easier to follow the standard. We are currently in Section 2.

In [717]:
# hex_digit_to_binary() converts a hex digit to binary
def hex_digit_to_binary(hex_digit):
    if hex_digit[:2] == "0x":
        hex_digit = hex_digit[2:]
    if len(hex_digit) > 1:
        return "ERROR: hex_digit_to_binary() expects a single hex digit"
    if hex_digit not in "0123456789abcdefABCDEF":
        return "ERROR: hex_digit_to_binary() expects a hex digit"
    # convert the hex digit to an integer
    hex_int = int(hex_digit, 16)

    # convert the integer to binary
    binary = bin(hex_int)

    # remove the "0b" from the binary string
    binary = binary[2:]

    # pad the binary string with zeros to make it 4-bits string
    binary = binary.zfill(4)
   

    return binary

# test the RFC example 
print("hex_digit_to_binary(\"7\") == 0111 ?", hex_digit_to_binary("7") == "0111")
print("hex_digit_to_binary(\"A\") == 1010 ?", hex_digit_to_binary("A") == "1010")
print("hex_digit_to_binary(\"a\") == 1010 ?", hex_digit_to_binary("a") == "1010")
print("hex_digit_to_binary(\"aa\") == 10101010 ?", hex_digit_to_binary("aa") == "10101010")

hex_digit_to_binary("7") == 0111 ? True
hex_digit_to_binary("A") == 1010 ? True
hex_digit_to_binary("a") == 1010 ? True
hex_digit_to_binary("aa") == 10101010 ? False


In [718]:
# create a helper function to print a binary string in groups of 4 bits
def split_after_n(string, n):
    substring =  [string[i:i+n] for i in range(0, len(string), n)]
    return " ".join(substring)

# hex_to_word() converts a hex string to binary
def hex_to_word(hex_string):
    # convert each hex digit to word
    word = ""
    for hex_digit in hex_string:
        word += hex_digit_to_binary(hex_digit)

    return word

def bytes_to_binary(b):
    binbytehs = ''.join(['{:08b}'.format(b) for b in bhs])
    return binbytehs

# is_word() checks if a string is a word
def _is_word(X,n=32, warning= False):
    # if it is not a string, it is not a word
    if not isinstance(X, str):
        return False
    
    # if it is not a binary string, it is not a word
    for x in X:
        if x != "0" and x != "1":
            return False
    
    # print warning message if the length is not equal to n
    if len(X) != n and warning:
        print("Warning: is_word() expected a string of length", n, "but got a string of length", len(X), "instead.")
    
    # otherwise, it is a word
    return True

def is_word(X,n=32, warning=False):
    # if X is a tuple, check if each element is a word
    if isinstance(X, tuple):
        for x in X:
            if not _is_word(x,n,warning):
                return False
        return True
    else:
        return _is_word(X,n,warning)

# test the RFC example 
print("hex_to_word(\"A103FE23\") == 1010 0001 0000 0011 1111 1110 0010 0011 ?", split_after_n(hex_to_word("A103FE23"),4) == "1010 0001 0000 0011 1111 1110 0010 0011")

# test bytes_to_binary()
hs = "A103FE23"
bhs = bytes.fromhex(hs)
print("bytes_to_binary(\"A103FE23\") == 1010 0001 0000 0011 1111 1110 0010 0011 ?", split_after_n(bytes_to_binary(bhs),4) == "1010 0001 0000 0011 1111 1110 0010 0011")

word = hex_to_word("A103FE23")
# test if equal to bytes_to_binary()
print("hex_to_word(\"A103FE23\") == bytes_to_binary(bytes.fromhex(\"A103FE23\")) ?", word == bytes_to_binary(bhs))

hex_to_word("A103FE23") == 1010 0001 0000 0011 1111 1110 0010 0011 ? True
bytes_to_binary("A103FE23") == 1010 0001 0000 0011 1111 1110 0010 0011 ? True
hex_to_word("A103FE23") == bytes_to_binary(bytes.fromhex("A103FE23")) ? True


In [719]:
def _integer_to_word_32(i):
    if i < pow(2, 32): # more efficient than 2 ** 32
        # convert the integer to hex 
        binary_version = bin(i)[2:] # remove the "0b" from the binary string

        # pad the binary string with zeros to make it 32 bits
        binary_version.zfill(32)

        return binary_version

def hex_repr(word):
    n = 4 # number of bits in a hex digit
    if is_word(word):
        substring =  [word[i:i+n] for i in range(0, len(word), n)] 
        hex_word = [hex(int(x, 2))[2:] for x in substring]
        return "".join(hex_word)

def integer_to_word(i):
    if i < pow(2, 32):
        return _integer_to_word_32(i).zfill(32)
    elif i < pow(2, 64):
        x = i // pow(2, 32)
        y = i % pow(2, 32)

        return (_integer_to_word_32(x).zfill(32) , _integer_to_word_32(y).zfill(32))
    else:
        raise ValueError("integer must be less than 2^64")

# test the RFC example
print(f"integer_to_word(291) == {integer_to_word(291)}")
print("hex_repr(integer_to_word(291)) == 00000123 ?", hex_repr(integer_to_word(291)) == "00000123")

integer_to_word(291) == 00000000000000000000000100100011
hex_repr(integer_to_word(291)) == 00000123 ? True


## Operations on words

We will use the following operations on words, we will use the notation from the standard to make it easier to follow the standard. We are currently in Section 3.

In [828]:
def _are_word_correct(X,Y):
    if not is_word(X) or not is_word(Y):
        raise ValueError(f"{X} and {Y} must be words")
    # if len(X) > 32 or len(Y) > 32:
    #     raise ValueError(f"{X} and {Y} must be less than 32 bits.\n X: {len(X)} bits, Y: {len(Y)} bits")
    return True
def _pad_words_32(X,Y):
    assert _are_word_correct(X,Y)
    # pad the shorter string with zeros
    if len(X) < len(Y):
        X = X.zfill(len(Y))
    elif len(Y) < len(X):
        Y = Y.zfill(len(X)) 
    return X,Y

def pad_words(X,Y):
    assert is_word(X) and is_word(Y) 
    if isinstance(X, tuple) and isinstance(Y, tuple):
        X,Y = _pad_words_32(X[0],Y[0]), _pad_words_32(X[1],Y[1])
        return X,Y
    elif isinstance(X, tuple) and  not isinstance(Y, tuple):
        Y.zfill(32)
        Y=("0" * 32, Y)
        return X,Y
    elif not isinstance(X, tuple) and  isinstance(Y, tuple):
        X.zfill(32)
        X=("0" * 32, X)
        return X,Y
    else:
        return _pad_words_32(X,Y)

def _bitwise_and_32(X,Y, n=32):
    assert _are_word_correct(X, Y)
    # X,Y = _pad_words_32(X, Y)
    result =  int(X,2) & int(Y,2)
    result_str = bin(result)[2:] # remove the "0b" from the binary string
    return result_str.zfill(n)

def bitwise_and(X,Y):
    assert is_word(X) and is_word(Y) # make sure the strings are words
    # X,Y = _pad_words_32(X, Y)
    X.zfill(32)
    Y.zfill(32)
    n = len(X)
    if isinstance(X, tuple) and isinstance(Y, tuple):
        return (_bitwise_and_32(X[0],Y[0]), _bitwise_and_32(X[1],Y[1]))
    elif isinstance(X, tuple) and  not isinstance(Y, tuple):
        raise ValueError("the first parameter is a tuple but the second is not")
    elif not isinstance(X, tuple) and  isinstance(Y, tuple):
        raise ValueError("the second parameter is a tuple but the first is not")
    else:
        return _bitwise_and_32(X,Y).zfill(n)


def _bitwise_or_32(X,Y):
    assert _are_word_correct(X, Y)
    # X,Y = _pad_words_32(X, Y)
    # n = len(X)
    # assert len(Y) == n # make sure the strings are the same length
    X.zfill(32)
    Y.zfill(32)
    n = len(X)
    result =  int(X,2) | int(Y,2)
    result_str = bin(result)[2:].zfill(n)
    return result_str

def bitwise_or(X,Y):
    assert is_word(X) and is_word(Y) # make sure the strings are words
    X,Y = pad_words(X,Y)
    n = len(X)

    if isinstance(X, tuple) and isinstance(Y, tuple):
        return (_bitwise_or_32(X[0],Y[0]), _bitwise_or_32(X[1],Y[1]))
    elif isinstance(X, tuple) and  not isinstance(Y, tuple):
        raise ValueError("the first parameter is a tuple but the second is not")
    elif not isinstance(X, tuple) and  isinstance(Y, tuple):
        raise ValueError("the second parameter is a tuple but the first is not")
    else:
        return _bitwise_or_32(X,Y).zfill(n)

def bitwise_not(X):
    assert is_word(X)  # make sure the strings are words
    s = ""
    for i in range(len(X)):
        if X[i] == "0":
            s += "1"
        else:
            s += "0"
    return s

def _bitwise_xor_32(X,Y):
    assert _are_word_correct(X, Y)
    # X,Y = _pad_words_32(X, Y)
    X.zfill(32)
    Y.zfill(32)
    n = len(X)
    result =  int(X,2) ^ int(Y,2)
    result_str = bin(result)[2:].zfill(32)
    return result_str.zfill(n)

def bitwise_xor(X,Y):
    assert is_word(X) and is_word(Y) # make sure the strings are words
    # X,Y = pad_words(X,Y)
    # n = len(X)

    # X,Y = _pad_words_32(X, Y)
    X.zfill(32)
    Y.zfill(32)
    n = len(X)
    if n != 32:
        raise ValueError("in the function bitwise_xor the parameters are not 32 bits long")
    if isinstance(X, tuple) and isinstance(Y, tuple):
        return (_bitwise_xor_32(X[0],Y[0]), _bitwise_xor_32(X[1],Y[1]))
    elif isinstance(X, tuple) and  not isinstance(Y, tuple):
        raise ValueError("the first parameter is a tuple but the second is not")
    elif not isinstance(X, tuple) and  isinstance(Y, tuple):
        raise ValueError("the second parameter is a tuple but the first is not")
    else:
        return _bitwise_xor_32(X,Y).zfill(n)

def plus_words(X,Y, n=32):
    if not is_word(X) or not is_word(Y):
        raise ValueError(f"the parameters {X} or {Y} are not words")
    if len(X) != len(Y):
        # issue a warning
        print(f"the parameters {X} and {Y} are not the same length. \n X is {len(X)} and Y is {len(Y)}")
    # X.zfill(32)
    # Y.zfill(32)
    # assert len(Y) == n and len(X) == n 

    result =  (int(X,2) + int(Y,2)) 
    result_str = integer_to_word(int(result)% pow(2,n))
    # print(f"result is {result} and result_str is {result_str}")
    return result_str.zfill(n)
        

a="01101100101110011101001001111011"
b="01101100101110011101001001111010"

print(f"a is {a} and b is {b}")
print(f"is a a word? {is_word(a)}")
print(f"is b a word? {is_word(b)}\n")
print("bitwise_and(a,b) == 01101100101110011101001001111010 ?", bitwise_and(a,b) == "01101100101110011101001001111010")
print("bitwise_or(a,b) == 01101100101110011101001001111011 ?", bitwise_or(a,b) == "01101100101110011101001001111011")
print("bitwise_not(a) == 10010011010001100010110110000100 ?", bitwise_not(a) == "10010011010001100010110110000100")
print("bitwise_xor(a,b) == 00000000000000000000000000000001 ?", bitwise_xor(a,b) == "00000000000000000000000000000001")

# test the RFC example
X = "01101100101110011101001001111011"
Y = "01100101110000010110100110110111"

print(f"\nX is {X} and Y is {Y}")

print("bitwise_xor(X,Y) == 00001001011110001011101111001100 ?", bitwise_xor(X,Y) == "00001001011110001011101111001100")

c = 2
d = 3

print(f"\nc is {c} and d is {d}")

c_str = integer_to_word(c)
d_str = integer_to_word(d)

print(f"c_str is {c_str} and d_str is {d_str}")

print(f"plus_words(c_str,d_str) == ", plus_words(c_str,d_str) )

print("------------------")

X = "1110000"
Y = "1101000"
n = len(X)
print(f"X is {X} and Y is {Y} and n is {n}")
print(f"plus_words(X,Y,n) == ", plus_words(X,Y,n=n) )

a is 01101100101110011101001001111011 and b is 01101100101110011101001001111010
is a a word? True
is b a word? True

bitwise_and(a,b) == 01101100101110011101001001111010 ? True
bitwise_or(a,b) == 01101100101110011101001001111011 ? True
bitwise_not(a) == 10010011010001100010110110000100 ? True
bitwise_xor(a,b) == 00000000000000000000000000000001 ? True

X is 01101100101110011101001001111011 and Y is 01100101110000010110100110110111
bitwise_xor(X,Y) == 00001001011110001011101111001100 ? True

c is 2 and d is 3
c_str is 00000000000000000000000000000010 and d_str is 00000000000000000000000000000011
plus_words(c_str,d_str) ==  00000000000000000000000000000101
------------------
X is 1110000 and Y is 1101000 and n is 7
plus_words(X,Y,n) ==  00000000000000000000000001011000


In [788]:
def SHR(X, n, w=32):
    if X[:2] == "0b":
        X = X[2:]
    assert is_word(X, w)
    w = len(X)
    n = n % w # make sure n is between 0 and w
    # x >> n
    res = "0" * n + X[:w-n]
    assert len(res) == w
    return res

def SHL(X, n, w=32):
    if X[:2] == "0b":
        X = X[2:]
    assert is_word(X, w)
    w = len(X)
    n = n % w # make sure n is between 0 and w
    # x << n
    res = X[n:] + "0" * n
    # res = X[:w-n] + "0" * n 
    assert len(res) == w
    return res

def ROTR(X, n):
    if X[:2] == "0b":
        X = X[2:]
    assert is_word(X) # make sure the strings are word
    w = len(X)
    if w != 32:
        raise ValueError(f"in ROTR the word is not 32 bits long. It is {w} bits long")
    n = n % w # make sure n is between 0 and w
    return bitwise_or(SHR(X,n), SHL(X,w-n))

def ROTL(X, n):
    if X[:2] == "0b":
        X = X[2:]
    assert is_word(X) # make sure the strings are word
    w = len(X)
    n = n % w # make sure n is between 0 and w
    return  bitwise_or(SHL(X,n), SHR(X,w-n))

In [722]:
a = int("011011001010011111111101011010101", 2)
bin_a = bin(a)[2:]
shifting = 1
print(f"a is {a}")
print(f"binary version of a is {bin_a}")
print(f"length of a is {len(bin_a)}")
print(f'SHR(a,shifting) == {SHR(bin(a), shifting)}')
print(f'SHL(a,shifting) == {SHL(bin(a), len(bin_a) -shifting)}')
print(f'ROTR(a,shifting) == {ROTR(bin(a), shifting)}')
print(f'ROTL(a,len(bin_a) -shifting) == {ROTL(bin(a), len(bin_a) - shifting)}')
print("ROTL(a,n) == ROTR(a,len(bin_a)-n) ?", ROTL(bin(a), shifting) == ROTR(bin(a), len(bin_a) - shifting))


a is 3645897429
binary version of a is 11011001010011111111101011010101
length of a is 32
SHR(a,shifting) == 01011001010011111111101011010101
SHL(a,shifting) == 10000000000000000000000000000000
ROTR(a,shifting) == 11011001010011111111101011010101
ROTL(a,len(bin_a) -shifting) == 11011001010011111111101011010101
ROTL(a,n) == ROTR(a,len(bin_a)-n) ? True


## Message Padding and Parsing

We are currently in Section 4. You see that there are different SHA functions, we will implement the message padding and parsing as described in the standard for SHA256. A good exercise for the reader would be to implement the other SHA functions.

Since we want to do things step by step, we will first implement with a specific example. We will use the example from the standard (section 4.1). We will then implement the function for a generic message in the next cell.

In [723]:
message = "0110000101100010011000110110010001100101"
L = len(message)
print(f"message is {message}")
print(f"length of message is {L}")
if L < pow(2,64):
    message = message + "1" 
    print(f"message is {message}")
    K = (448 - L - 1)  % 512
    print(f"K is {K}")
    message = message + "0" * K
    print(f"message is {split_after_n(hex_repr(message),8)}")

    L_bin = integer_to_word(L).zfill(64)
    print(f"L is {L_bin}")

    message = message + L_bin

    print(f"message is {split_after_n(hex_repr(message),8)}")

message is 0110000101100010011000110110010001100101
length of message is 40
message is 01100001011000100110001101100100011001011
K is 407
message is 61626364 65800000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
L is 0000000000000000000000000000000000000000000000000000000000101000
message is 61626364 65800000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000028


In [810]:
def pad_message(message):
    L = len(message)
    if L < pow(2,64):
        message = message + "1" 
        K = (448 - L - 1)  % 512
        message = message + "0" * K
        L_bin = integer_to_word(L).zfill(64)
        message = message + L_bin
    return message

message = "0110000101100010011000110110010001100101" 
print(f"pad_message({message}) == \n{split_after_n(hex_repr(pad_message(message)),8)}")

# this is done to see why you need a 1 when you pad the message
message_new = "011000010110001001100011011001000110010100"
print(f"pad_message({message_new}) == \n{split_after_n(hex_repr(pad_message(message_new)),8)}")

pad_message(0110000101100010011000110110010001100101) == 
61626364 65800000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000028
pad_message(011000010110001001100011011001000110010100) == 
61626364 65200000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 0000002a


## Functions and Constants Used

We are currently in Section 5. We will implement the functions and constants used in the standard. Again, we are doing this for SHA256, but the same ideas are used for the other SHA functions.

In [725]:
def CH(X,Y,Z):
    if isinstance(X, tuple):
        print(f"X is {X}")
    res = bitwise_xor(bitwise_and(X,Y), bitwise_and(bitwise_not(X),Z))
    # print(f"CH({X},{Y},{Z}) == {res}")
    return res

def MAJ(X,Y,Z):
    if isinstance(X, tuple):
        print(f"X is {X}")
    res = bitwise_xor(bitwise_xor(bitwise_and(X,Y), bitwise_and(X,Z)), bitwise_and(Y,Z))
    # print(f"MAJ({X},{Y},{Z}) == {res}")
    return res

def BSIG0(X):
    if isinstance(X, tuple):
        print(f"X is {X}")
    res = bitwise_xor(bitwise_xor(ROTR(X,2), ROTR(X,13)), ROTR(X,22))
    # print(f"BSIG0({X}) == {res}")
    return res

def BSIG1(X):
    if isinstance(X, tuple):
        print(f"X is {X}")
    
    rotr6 = ROTR(X,6)
    rotr11 = ROTR(X,11)
    res1 = bitwise_xor(rotr6, rotr11)

    res2 = bitwise_xor(res1, ROTR(X,25))
    return res2

def SSIG0(X):
    if isinstance(X, tuple):
        print(f"X is {X}")
    res = bitwise_xor(bitwise_xor(ROTR(X,7), ROTR(X,18)), SHR(X,3))
    # print(f"SSIG0({X}) == {res}")
    return res

def SSIG1(X):
    if isinstance(X, tuple):
        print(f"X is {X}")
    res = bitwise_xor(bitwise_xor(ROTR(X,17), ROTR(X,19)), SHR(X,10))
    # print(f"SSIG1({X}) == {res}")
    if len(res) != 32:
        raise ValueError(f"SSIG1({X}) == {res} is not 32 bits long")
    return res

## Computing the Message Digest

We are currently in Section 6. We will implement the functions and constants used in the standard. Again, we are doing this for SHA256, but the same ideas are used for the other SHA functions.

The following *initial hash value* is used in the standard. It is derived from the first 32 bits of the fractional parts of the square roots of the first eight prime numbers. (the other constants were derived from the first 32 bits of the fractional parts of the *cube* roots of the first sixty-four prime numbers).

to process the message we assume it is in hexadecimal format. We will first convert it to binary format. We will then pad the message and split it into 512 bit blocks. We will then process each block.

In [833]:
def hash_processing(message, print_updates=False):

    const_hex = ["428a2f98","71374491","b5c0fbcf","e9b5dba5","3956c25b","59f111f1","923f82a4","ab1c5ed5","d807aa98","12835b01","243185be","550c7dc3","72be5d74","80deb1fe","9bdc06a7","c19bf174","e49b69c1","efbe4786","0fc19dc6","240ca1cc","2de92c6f","4a7484aa","5cb0a9dc","76f988da","983e5152","a831c66d","b00327c8","bf597fc7","c6e00bf3","d5a79147","06ca6351","14292967","27b70a85","2e1b2138","4d2c6dfc","53380d13","650a7354","766a0abb","81c2c92e","92722c85","a2bfe8a1","a81a664b","c24b8b70","c76c51a3","d192e819","d6990624","f40e3585","106aa070","19a4c116","1e376c08","2748774c","34b0bcb5","391c0cb3","4ed8aa4a","5b9cca4f","682e6ff3","748f82ee","78a5636f","84c87814","8cc70208","90befffa","a4506ceb","bef9a3f7","c67178f2"]
    const = [hex_to_word(hex_string=const_hex[i]) for i in range(0, len(const_hex))]


    H0 = ["6a09e667","bb67ae85","3c6ef372","a54ff53a","510e527f","9b05688c","1f83d9ab","5be0cd19"]
    H = [hex_to_word(hex_string=H0[i]) for i in range(0, len(H0))]
    Hf = H.copy()
    # for i in range(0, len(H)):
    #     if len(H[i]) != 32:
    #         raise ValueError(f"H[{i}] is not 32 bits long. It is {len(H[i])} bits long")
    
    # if message is hex string, convert to binary
    if isinstance(message, str) and all(c in '0123456789abcdef' for c in message):
        message = hex_to_word(message)
    # if message is bytes string, convert to binary
    elif isinstance(message, bytes):
        message = message.hex()
        message = hex_to_word(message)
        # message = hex_to_word(hex_string=message)
    elif isinstance(message, str) and all(c in '01' for c in message):
        message = message
    else:
        raise ValueError("Error: message must be hex string, bytes string, or binary string")

    # print(f"prepadded_message is {message}")
    padded_message = pad_message(message)
    # print(f"padded_message is {padded_message}")
    block_length = 512 # bits
    messages = [padded_message[i:i+block_length] for i in range(0, len(padded_message), block_length)]
    hex_messages = [hex_repr(word=messages[i]) for i in range(0, len(messages))]
    # print(f"hex_messages is {hex_messages[0]}")
    # print(f"len(hex_messages) is {len(hex_messages[0])}")
    # assert len(messages) == len(padded_message) / 512, "Error: len(messages) != padded_message / 512"
    # print(f"messages is {messages}")
    # print(f"len(messages) is {len(messages)}")

    W = ['0'] * 64
    for i in range(0, len(messages)):
        curr_message = messages[i] # 512 bits
        # len_curr_message = len(curr_message)
        # print(f"len_curr_message is {len_curr_message}")
        sub_curr_message = [curr_message[j:j+32] for j in range(0, len(curr_message), 32)]
        # len_sub_curr_message = len(sub_curr_message)
        # print(f"len_sub_curr_message is {len_sub_curr_message}")
        # print(f"len(sub_curr_message) is {len(sub_curr_message)}")

        # 1. Prepare the message schedule
        for t in range(0, 16):
            W[t] = sub_curr_message[t]
        for t in range(16, 64):
            sum1 = plus_words(SSIG1(W[t-2]),W[t-7]) # all added words are 32 bits, or equivalently addition is performed modulo 2^32
            sum2 = plus_words(SSIG0(W[t-15]), W[t-16])
            W[t] = plus_words(sum1, sum2)

        # 2. Initialize the eight working variables
        a = H[0]
        b = H[1]
        c = H[2]
        d = H[3]
        e = H[4]
        f = H[5]
        g = H[6]
        h = H[7]

        # 3. Perform the main hash computation
        for t in range(0, 64):
            T1 = plus_words(plus_words(plus_words(plus_words(h, BSIG1(e)).zfill(32), CH(e,f,g)).zfill(32), const[t]).zfill(32), W[t]).zfill(32)
            T2 = plus_words(BSIG0(a), MAJ(a,b,c)).zfill(32)
            h = g
            g = f
            f = e
            e = plus_words(d, T1).zfill(32)
            d = c
            c = b
            b = a
            a = plus_words(T1, T2).zfill(32)

        # 4. Compute the intermediate hash value
        H[0] = plus_words(a, H[0])
        H[1] = plus_words(b, H[1])
        H[2] = plus_words(c, H[2])
        H[3] = plus_words(d, H[3])
        H[4] = plus_words(e, H[4])
        H[5] = plus_words(f, H[5])
        H[6] = plus_words(g, H[6])
        H[7] = plus_words(h, H[7])

    # 5. Produce the final hash value
        final_hash = "".join(H)
        if print_updates:
            print(f"current_hash is {final_hash}")


    return hex_repr(final_hash) 




# message = bytes.fromhex(str(message))
# print("\nmy func is (bytes)",hash_processing(message),sep='\t')
# print("hashlib fromhex     ",hashlib.sha256(message).hexdigest(),sep='\t') 
# print(f"are they equal? {hash_processing(message) == hashlib.sha256(message).hexdigest()}")



## Testing the Hash Function

We will now test the hash function. We will use the test vector created in the beginning of this notebook. We will compare the result of our function with the result of the hashlib library.

In [836]:
for i in range(1,4):
    pre_image = preimage_test[i].hex()
    print(f"pre_image is {pre_image}")
    print(f"hash_processing(pre_image) is\t{hash_processing(pre_image)}")
    print(f"while  OTOH   hash_test[i] is\t{hashlib.sha256(bytes.fromhex(pre_image)).hexdigest()}")
    print(f"hash_processing(pre_image) == hash_test[{i}] ? {hash_processing(pre_image) == hash_test[i]}")
    print("\n")

pre_image is 906a2ca2051bbb2d29eb
hash_processing(pre_image) is	3e9009fcc7b713addcb737a27d83e8659fd34bd17325513207e75995aac29f1c
while  OTOH   hash_test[i] is	3e9009fcc7b713addcb737a27d83e8659fd34bd17325513207e75995aac29f1c
hash_processing(pre_image) == hash_test[1] ? True


pre_image is e5c927061df2ee76538620b289c88fa6c91f1dbe
hash_processing(pre_image) is	56e1cea6b40118cd1ecfae70cf29d275bc9bb2b45e6992739d7892e993482134
while  OTOH   hash_test[i] is	56e1cea6b40118cd1ecfae70cf29d275bc9bb2b45e6992739d7892e993482134
hash_processing(pre_image) == hash_test[2] ? True


pre_image is 4f64b6a3d046ddb79a366c4c0b2d97b417d2c55280cd9b8ca544a295d2bf
hash_processing(pre_image) is	0e5e2bb5390933c316000192e672cbfcf40a59e2f35bf52c39b79b5ff7e0e5d4
while  OTOH   hash_test[i] is	0e5e2bb5390933c316000192e672cbfcf40a59e2f35bf52c39b79b5ff7e0e5d4
hash_processing(pre_image) == hash_test[3] ? True


